# **ML Assignment-5**


Chinni Vamshi Krushna 2021A7PS2084H

Atharva Chikhale 2021A7PS2752H

Rishabh Mittal 2021A7PS2620H

In [1]:
# !pip show torch
# !pip install torch

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler


In [3]:
df= pd.read_csv("/content/abalone.data.csv")

In [4]:
#Encoding column of Gender
data = pd.get_dummies(df, columns=['Sex'])

# Standardize the continuous features
continuous_features = ['Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight']
scaler = StandardScaler()
data[continuous_features] = scaler.fit_transform(data[continuous_features])

# Display the preprocessed data
print(data.head())

     Length  Diameter    Height  Whole_weight  Shucked_weight  Viscera_weight  \
0 -0.574558 -0.432149 -1.064424     -0.641898       -0.607685       -0.726212   
1 -1.448986 -1.439929 -1.183978     -1.230277       -1.170910       -1.205221   
2  0.050033  0.122130 -0.107991     -0.309469       -0.463500       -0.356690   
3 -0.699476 -0.432149 -0.347099     -0.637819       -0.648238       -0.607600   
4 -1.615544 -1.540707 -1.423087     -1.272086       -1.215968       -1.287337   

   Shell_weight  Rings  Sex_F  Sex_I  Sex_M  
0     -0.638217     15  False  False   True  
1     -1.212987      7  False  False   True  
2     -0.207139      9   True  False  False  
3     -0.602294     10  False  False   True  
4     -1.320757      7  False   True  False  


In [5]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['Rings'])
y = data['Rings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

X_train.dropna(inplace=True)
X_test.dropna(inplace=True)
#BOOL to INT
X_train[['Sex_F', 'Sex_I', 'Sex_M']] = X_train[['Sex_F', 'Sex_I', 'Sex_M']].astype(int)
X_test[['Sex_F', 'Sex_I', 'Sex_M']] = X_test[['Sex_F', 'Sex_I', 'Sex_M']].astype(int)

# Convert into PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


In [6]:
import torch.nn as nn

class AbaloneModel(nn.Module):
    def __init__(self, hidden_nodes1=64, hidden_nodes2=32):
        super(AbaloneModel, self).__init__()
        self.fc1 = nn.Linear(10, hidden_nodes1)  # Input layer: 10 nodes
        self.fc2 = nn.Linear(hidden_nodes1, hidden_nodes2)  # First hidden layer
        self.fc3 = nn.Linear(hidden_nodes2, 1)    # Output layer: 1 node for regression
        self.hidden_nodes1 = hidden_nodes1
        self.hidden_nodes2 = hidden_nodes2

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))  # ReLU activation for the first layer
        x = nn.functional.relu(self.fc2(x))  # ReLU activation for the second layer
        x = self.fc3(x)  # Output layer
        return x


In [8]:
import torch.optim as optim


def train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, epochs=100, batch_size=32):
    num_batches = len(X_train_tensor) // batch_size

    for epoch in range(epochs):
        indices = torch.randperm(len(X_train_tensor))
        X_train_tensor_shuffled = X_train_tensor[indices]
        y_train_tensor_shuffled = y_train_tensor[indices]

        for i in range(0, len(X_train_tensor), batch_size):
            # Getbatch
            X_batch = X_train_tensor_shuffled[i:i+batch_size]
            y_batch = y_train_tensor_shuffled[i:i+batch_size]

            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Print loss after each epoch
        # if (epoch+1) % 10 == 0:
        #     print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


def evaluate(model, X_test_tensor, y_test_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        mse = nn.MSELoss()
        loss = mse(outputs, y_test_tensor)
        # print(f'Test Loss: {loss.item():.4f}')
        return loss.item()

# Define the model
model = AbaloneModel()

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor)

# Evaluate the model
mse = evaluate(model, X_test_tensor, y_test_tensor)
print(f"Test loss for 100 epochs with 0.01 lr: {mse}")


Test loss for 100 epochs with 0.01 lr: 4.473552703857422


In [9]:
import torch.optim as optim
import pandas as pd
#Various Hyper-Params
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]
hidden_nodes1 = [32, 64, 128]
hidden_nodes2= [32,64,128]
hidden_layers = [1, 2, 3]
epoch_sizes=[20,40,60,80,100,125]
result_lr = []

for lr in learning_rates:
  for e in epoch_sizes:
    for batchs in batch_sizes:
      for nodes1 in hidden_nodes1:
        for nodes2 in hidden_nodes2:
    # Define and train the model
          model = AbaloneModel(hidden_nodes1=nodes1,hidden_nodes2= nodes2)
          optimizer = optim.SGD(model.parameters(), lr=lr)
          train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, epochs=e, batch_size= batchs)

          # Evaluate the model
          mse = evaluate(model, X_test_tensor, y_test_tensor)

          # Store the results
          result_lr.append({
                    'Learning Rate': lr,
                    'Batch Size': batchs,
                    'Hidden Nodes in Layer 1': nodes1,
                    'Hidden Nodes in Layer2': nodes2,
                    'Epochs': e,
                    'MSE': mse
          })

# Print the results
# for result in result_lr:
    # print(f"Learning Rate: {result['Learning Rate']}, MSE: {result['MSE']}")

In [11]:
print(result_lr)

[{'Learning Rate': 0.001, 'Batch Size': 16, 'Hidden Nodes in Layer 1': 32, 'Hidden Nodes in Layer2': 32, 'Epochs': 20, 'MSE': 4.564385890960693}, {'Learning Rate': 0.001, 'Batch Size': 16, 'Hidden Nodes in Layer 1': 32, 'Hidden Nodes in Layer2': 64, 'Epochs': 20, 'MSE': 4.605739116668701}, {'Learning Rate': 0.001, 'Batch Size': 16, 'Hidden Nodes in Layer 1': 32, 'Hidden Nodes in Layer2': 128, 'Epochs': 20, 'MSE': 4.724771022796631}, {'Learning Rate': 0.001, 'Batch Size': 16, 'Hidden Nodes in Layer 1': 64, 'Hidden Nodes in Layer2': 32, 'Epochs': 20, 'MSE': 4.666952610015869}, {'Learning Rate': 0.001, 'Batch Size': 16, 'Hidden Nodes in Layer 1': 64, 'Hidden Nodes in Layer2': 64, 'Epochs': 20, 'MSE': 4.547297954559326}, {'Learning Rate': 0.001, 'Batch Size': 16, 'Hidden Nodes in Layer 1': 64, 'Hidden Nodes in Layer2': 128, 'Epochs': 20, 'MSE': 4.708538055419922}, {'Learning Rate': 0.001, 'Batch Size': 16, 'Hidden Nodes in Layer 1': 128, 'Hidden Nodes in Layer2': 32, 'Epochs': 20, 'MSE': 4

In [12]:
results_df = pd.DataFrame(result_lr)

# Save the DataFrame to a CSV file
results_df.to_csv('All_results.csv', index=False)


In [16]:
top_10_lowest_mse = results_df.nsmallest(5, 'MSE')
print("Best results")
print(top_10_lowest_mse)
# Top 10 rows with highest 'MSE'
top_10_highest_mse = results_df.nlargest(10, 'MSE')


Best results
     Learning Rate  Batch Size  Hidden Nodes in Layer 1  \
253          0.010          32                       32   
243          0.010          16                       32   
136          0.001          16                       32   
216          0.010          16                       32   
226          0.010          32                       32   

     Hidden Nodes in Layer2  Epochs       MSE  
253                      64      80  4.293335  
243                      32      80  4.297072  
136                      64     125  4.325491  
216                      32      60  4.326936  
226                      64      60  4.327290  


##TASK 7
### Guiding through the hyper parameters
For this we changed various hyper params simultaneously, ie No. of epochs, learning rate, batch size, the number of hidden nodes in a layer.

The detailed results can be downloaded from All_results.csv file or results_lr.
To explain in brief-
Optimizer- SGD
1. Learning rate- 0.1 was too high and gave really high loss, 0.01 was a ideal number
2. Number of epochs- Around 80 lead to best results, after that there seems to be overfitting for 0.01 lr
3. Batch size- 32 or 64, lower batch size led to underfitting
4. No. of nodes in a layer- Since we had 2 hidden layers, we tried different combinations
Having (32,64) ked to best results... Lower values led to UNDERFITTING

###TASK 8
##Part 1
###Changing no. of hidden layers in hyper params

##Part 2
###Using ‘Adagrad’ as optimizer for the model

In [ ]:
#SGD TO ADAGRAD
import torch.optim as optim
import pandas as pd

learning_rates = [0.01, 0.1]
batch_sizes = [32]
hidden_nodes1 = [32]
hidden_nodes2= [32,64]
# hidden_layers = [1, 2, 3]
epoch_sizes=[60, 100, 125]
result_lr_Adagrad = []

for lr in learning_rates:
  for e in epoch_sizes:
    for batchs in batch_sizes:
      for nodes1 in hidden_nodes1:
        for nodes2 in hidden_nodes2:
    # Define and train the model
          model = AbaloneModel(hidden_nodes1=nodes1,hidden_nodes2= nodes2)
          optimizer = optim.Adagrad(model.parameters(), lr=lr)
          train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, epochs=e, batch_size= batchs)

          # Evaluate the model
          mse = evaluate(model, X_test_tensor, y_test_tensor)

          # Store the results
          result_lr_Adagrad.append({
                    'Learning Rate': lr,
                    'Batch Size': batchs,
                    'Hidden Nodes in Layer 1': nodes1,
                    'Hidden Nodes in Layer2': nodes2,
                    'Epochs': e,
                    'MSE': mse
          })

# Print the results
# for result in result_lr:
    # print(f"Learning Rate: {result['Learning Rate']}, MSE: {result['MSE']}")

In [20]:
#Result comparisons
results_df2 = pd.DataFrame(result_lr_Adagrad)
print("Best results SGD vs Adagrad")

results_df2.to_csv('All_results_Ada.csv', index=False)
top_10_lowest_mse_Ada = results_df2.nsmallest(5, 'MSE')
print("SGD")
print(top_10_lowest_mse)
print("AdaGrad")
print(top_10_lowest_mse_Ada)


Best results SGD vs Adagrad
SGD
     Learning Rate  Batch Size  Hidden Nodes in Layer 1  \
253          0.010          32                       32   
243          0.010          16                       32   
136          0.001          16                       32   
216          0.010          16                       32   
226          0.010          32                       32   

     Hidden Nodes in Layer2  Epochs       MSE  
253                      64      80  4.293335  
243                      32      80  4.297072  
136                      64     125  4.325491  
216                      32      60  4.326936  
226                      64      60  4.327290  
AdaGrad
    Learning Rate  Batch Size  Hidden Nodes in Layer 1  \
6             0.1          32                       32   
11            0.1          32                       32   
9             0.1          32                       32   
10            0.1          32                       32   
7             0.1          

From above it's clear that Adagrad and SGD provided simialar results but SGD was a bit worse in this case.
but ADAGRAD appeared to be "MUCH FASTER THAN SGD"

Observations:
1)Both optimizers seem to converge to similar final MSE values, with Adagrad slightly outperforming SGD in terms of final MSE.

2)Adagrad tends to have a more stable loss curve over epochs compared to SGD. This can be seen from the fact that the loss for Adagrad remains constant at 0.9895 after a certain number of epochs, while the loss for SGD fluctuates slightly.

3)Adagrad adjusts the learning rates for each parameter individually based on the historical gradient values, which can be beneficial especially for sparse data or data with high dimensionality. This adaptiveness might contribute to its slightly better performance in this scenario compared to the fixed learning rate of SGD.

4)However, it's worth noting that Adagrad can have issues with overly aggressive learning rate decay, which might hinder its performance in some cases.

Overall, while Adagrad shows a marginal improvement in terms of MSE compared to SGD in this specific scenario, the choice between the two optimizers might depend on various factors such as the specific characteristics of the dataset, computational resources, and training time.

Now coming onto Hidden Layers

In [52]:
import torch.nn as nn

class AbaloneModelNew(nn.Module):
    def __init__(self, hidden_nodes1=64, hidden_nodes2=32):
        super(AbaloneModelNew, self).__init__()
        self.fc1 = nn.Linear(10, hidden_nodes1)  # Input layer: 10 nodes

        # Hidden layers
        self.fc2 = nn.Linear(hidden_nodes1, hidden_nodes2)
        self.fc3 = nn.Linear(hidden_nodes2, hidden_nodes2)
        self.fc4 = nn.Linear(hidden_nodes2, hidden_nodes2)
        self.fc5 = nn.Linear(hidden_nodes2, hidden_nodes2)
        self.fc6 = nn.Linear(hidden_nodes2, hidden_nodes2)
        self.fc7 = nn.Linear(hidden_nodes2, hidden_nodes2)
        self.fc8 = nn.Linear(hidden_nodes2, hidden_nodes2)
        self.fc9 = nn.Linear(hidden_nodes2, hidden_nodes2)
        # self.fc10 = nn.Linear(hidden_nodes2, hidden_nodes2)
        # self.fc11 = nn.Linear(hidden_nodes2, hidden_nodes2)
        self.fc12 = nn.Linear(hidden_nodes2, hidden_nodes2)

        self.fc13 = nn.Linear(hidden_nodes2, 1)    # Output layer: 1 node for regression

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))  # ReLU activation for the first layer
        x = nn.functional.relu(self.fc2(x))  # ReLU activation for the second layer
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = nn.functional.relu(self.fc5(x))
        x = nn.functional.relu(self.fc6(x))
        x = nn.functional.relu(self.fc7(x))
        x = nn.functional.relu(self.fc8(x))
        x = nn.functional.relu(self.fc9(x))
        # x = nn.functional.relu(self.fc10(x))
        # x = nn.functional.relu(self.fc11(x))
        x = nn.functional.relu(self.fc12(x))
        x = self.fc13(x)  # Output layer
        return x

# # Create an instance of the AbaloneModel class
# model = AbaloneModelNew()

# # Print the model architecture
# print(model)


In [53]:
import torch.optim as optim
import pandas as pd

learning_rates = [0.01, 0.1]
batch_sizes = [32]
hidden_nodes1 = [32]
hidden_nodes2= [32,64]
# hidden_layers = [1, 2, 3]
epoch_sizes=[60, 75, 100, 125]
result_lr_hid = []

for lr in learning_rates:
  for e in epoch_sizes:
    for batchs in batch_sizes:
      for nodes1 in hidden_nodes1:
        for nodes2 in hidden_nodes2:
          model = AbaloneModelNew(hidden_nodes1=nodes1,hidden_nodes2= nodes2)
          optimizer = optim.SGD(model.parameters(), lr=lr)
          train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, epochs=e, batch_size= batchs)

          mse = evaluate(model, X_test_tensor, y_test_tensor)

          # Store the results
          result_lr_hid.append({
                    'Learning Rate': lr,
                    'Batch Size': batchs,
                    'Hidden Nodes in Layer 1': nodes1,
                    'Hidden Nodes in Layer2': nodes2,
                    'Epochs': e,
                    'MSE': mse
          })

# Print the results
# for result in result_lr:
    # print(f"Learning Rate: {result['Learning Rate']}, MSE: {result['MSE']}")

In [54]:
results_df_hidden = pd.DataFrame(result_lr_hid)
top_5_lowest_mse_hid = results_df_hidden.nsmallest(5, 'MSE')
print("Results in 10+ hidden layers")
print(top_5_lowest_mse_hid)


Results in 10+ hidden layers
   Learning Rate  Batch Size  Hidden Nodes in Layer 1  Hidden Nodes in Layer2  \
7           0.01          32                       32                      64   
5           0.01          32                       32                      64   
1           0.01          32                       32                      64   
3           0.01          32                       32                      64   
2           0.01          32                       32                      32   

   Epochs       MSE  
7     125  4.723125  
5     100  4.784381  
1      60  4.922831  
3      75  5.003715  
2      75  5.080157  


In general, changing the size of hidden layers can significantly impact the overall accuracy, speed, and power of a given neural network system.
But here it's similar because of less complexity in data.

#### Using Sigmoid function

In [56]:
import torch.nn as nn

class AbaloneModelSig(nn.Module):
    def __init__(self, hidden_nodes1=64, hidden_nodes2=32):
        super(AbaloneModelSig, self).__init__()
        self.fc1 = nn.Linear(10, hidden_nodes1)  # Input layer: 10 nodes
        self.fc2 = nn.Linear(hidden_nodes1, hidden_nodes2)  # First hidden layer
        self.fc3 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Second hidden layer
        self.fc4 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Third hidden layer
        self.fc5 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Fourth hidden layer
        self.fc6 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Fifth hidden layer
        self.fc7 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Sixth hidden layer
        self.fc8 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Seventh hidden layer
        self.fc9 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Eighth hidden layer
        self.fc10 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Ninth hidden layer
        self.fc11 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Tenth hidden layer
        self.fc12 = nn.Linear(hidden_nodes2, hidden_nodes2)  # Eleventh hidden layer
        self.fc13 = nn.Linear(hidden_nodes2, 1)    # Output layer: 1 node for regression

    def forward(self, x):
        x = nn.functional.sigmoid(self.fc1(x))  # Sigmoid activation for the first layer
        x = nn.functional.sigmoid(self.fc2(x))  # Sigmoid activation for the second layer
        x = nn.functional.sigmoid(self.fc3(x))  # Sigmoid activation for the third layer
        x = nn.functional.sigmoid(self.fc4(x))  # Sigmoid activation for the fourth layer
        x = nn.functional.sigmoid(self.fc5(x))  # Sigmoid activation for the fifth layer
        x = nn.functional.sigmoid(self.fc6(x))  # Sigmoid activation for the sixth layer
        x = nn.functional.sigmoid(self.fc7(x))  # Sigmoid activation for the seventh layer
        x = nn.functional.sigmoid(self.fc8(x))  # Sigmoid activation for the eighth layer
        x = nn.functional.sigmoid(self.fc9(x))  # Sigmoid activation for the ninth layer
        x = nn.functional.sigmoid(self.fc10(x))  # Sigmoid activation for the tenth layer
        x = nn.functional.sigmoid(self.fc11(x))  # Sigmoid activation for the eleventh layer
        x = nn.functional.sigmoid(self.fc12(x))  # Sigmoid activation for the twelfth layer
        x = self.fc13(x)  # Output layer
        return x

# Create an instance of the AbaloneModel class
model = AbaloneModelSig()

# Print the model architecture
print(model)


AbaloneModelSig(
  (fc1): Linear(in_features=10, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=32, bias=True)
  (fc6): Linear(in_features=32, out_features=32, bias=True)
  (fc7): Linear(in_features=32, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=32, bias=True)
  (fc9): Linear(in_features=32, out_features=32, bias=True)
  (fc10): Linear(in_features=32, out_features=32, bias=True)
  (fc11): Linear(in_features=32, out_features=32, bias=True)
  (fc12): Linear(in_features=32, out_features=32, bias=True)
  (fc13): Linear(in_features=32, out_features=1, bias=True)
)


In [57]:
import torch.optim as optim
import pandas as pd

learning_rates = [0.01, 0.1]
batch_sizes = [32]
hidden_nodes1 = [32]
hidden_nodes2= [64]
# hidden_layers = [1, 2, 3]
epoch_sizes=[100]
result_lr_sig = []

for lr in learning_rates:
  for e in epoch_sizes:
    for batchs in batch_sizes:
      for nodes1 in hidden_nodes1:
        for nodes2 in hidden_nodes2:
          model = AbaloneModelSig(hidden_nodes1=nodes1,hidden_nodes2= nodes2)
          optimizer = optim.SGD(model.parameters(), lr=lr)
          train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, epochs=e, batch_size= batchs)

          mse = evaluate(model, X_test_tensor, y_test_tensor)

          # Store the results
          result_lr_sig.append({
                    'Learning Rate': lr,
                    'Batch Size': batchs,
                    'Hidden Nodes in Layer 1': nodes1,
                    'Hidden Nodes in Layer2': nodes2,
                    'Epochs': e,
                    'MSE': mse
          })

# Print the results
# for result in result_lr:
    # print(f"Learning Rate: {result['Learning Rate']}, MSE: {result['MSE']}")

In [59]:
results_df_hidd = pd.DataFrame(result_lr_sig)
top_5_lowest_mse_h = results_df_hidd.nsmallest(5, 'MSE')
print("Results in 10+ hidden layers using Sigmoid(vanishing gradient).")
print(top_5_lowest_mse_h)


Results in 10+ hidden layers
   Learning Rate  Batch Size  Hidden Nodes in Layer 1  Hidden Nodes in Layer2  \
0           0.01          32                       32                      64   
1           0.10          32                       32                      64   

   Epochs        MSE  
0     100  10.837725  
1     100  10.862483  


##Observations
On using sigmoid instead of relu for 10+ hidden layers, loss significantly increased.

1) Vanishing Gradient Problem:

The loss seems to stagnate at around 10.83 after a certain number of epochs and does not decrease further. This can indicate that gradients might be vanishing as they propagate backward through the network during training. The vanishing gradient problem occurs when gradients become extremely small as they are backpropagated through many layers with saturating activation functions like Sigmoid. As a result, the weights in the earlier layers are updated very slowly, if at all, leading to slow or no learning.

2) Comparison with Previous Architectures:

When comparing the final MSE on the test set:

SGD optimizer:4.473552703857422

Adagrad optimizer: 4.435867

More hidden layers with ReLU activation: 4.723125 

More hidden layers with Sigmoid activation: 10.837725

In terms of minimizing the loss, the earlier architectures with either SGD or Adagrad optimizers might perform better. Both architectures seem to converge to a lower final MSE on the test set compared to the model with increased hidden layers and Sigmoid activation function.
While the model with increased hidden layers and Sigmoid activation function may have the capacity to learn complex patterns, it appears to suffer from the vanishing gradient problem, which limits its ability to effectively optimize and minimize the loss.
In summary, while increasing the number of hidden layers and using Sigmoid activation functions can potentially capture more complex relationships in the data, it's crucial to address issues like vanishing gradients to ensure effective training and improved performance. In this case, the architectures with simpler configurations and different optimizers (SGD or Adagrad) seem to perform better in terms of minimizing the loss.